<a href="https://colab.research.google.com/github/JoyeBright/Multilingual_Multimodal_Universal_MT/blob/main/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
path = "/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Per_Eng_Trans_V1/raw_data/TEP"

## **Sentence-BERT**

In [23]:
import numpy as np

In [24]:
!pip3 install torch torchvision

In [25]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.9244, 0.8952, 0.3670],
        [0.2733, 0.3381, 0.1173],
        [0.6424, 0.0800, 0.1753],
        [0.4000, 0.1564, 0.1065],
        [0.9632, 0.6136, 0.0193]])


In [27]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.4.1.2)


In [28]:
train_src = path+"/train.tok.clean.src"

In [29]:
!head -10 $train_src

یون سنگ، تو میری از او بپرسی چیزی پیدا کرده یا نه؟
چيز غيرعادي اي نبود .
...البته کاملا مزه گوشت گاو را نداره
شماها باید به این خاتمه بدین .
بله. اما او به نظر آشفته بود. مشکلی پیش اومده؟
ولي خوني شده .
شيطنت کردن .
اما اگر عميقتر ببينيد برتر از اين هم وجود داره، يعني زير زمين
معنی سادش اینه که شما میخواید فرار کنید .
باقيماندن .


In [30]:
from sentence_transformers import SentenceTransformer
name = 'https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/xlm-r-100langs-bert-base-nli-mean-tokens.zip'

In [31]:
model = SentenceTransformer(model_name_or_path=name, device='cuda')

In [32]:
with open(train_src) as f:
    content = f.readlines()
    content = [x.strip() for x in content]

In [33]:
sentences = content

In [40]:
sentence_embeddings = model.encode(sentences=sentences, show_progress_bar=True, convert_to_numpy=True)

In [41]:
print(type(sentence_embeddings))
print(sentence_embeddings.shape)

<class 'numpy.ndarray'>
(610086, 768)


In [42]:
print(sentence_embeddings[:1])

[[ 0.4593516  -0.01457303  1.7394984   0.4404026   0.6362638   0.2528962
   0.11195075  0.2921515   0.10742761  0.15947405 -0.21293342  0.5458668
   0.2740503   0.5479267  -0.32321924  0.13399549  0.04454777 -0.57006615
   0.18741901 -0.5034281   0.17253752  0.3391332   0.10633039 -0.69341135
   0.30869687 -0.06997975 -0.12753329 -0.82400626 -0.6240134   0.20505981
   0.03349856 -0.04983833 -0.27879694 -0.17779048 -0.898158    0.9478081
   0.7969705   0.10409191  0.04230267  0.0060135   0.0042068   0.06179347
   0.30250612  0.8499539  -0.9323933  -0.5255916   0.36498743  0.12332773
  -0.19779778 -1.1141446  -0.5156098   0.08481535  0.55320185  0.39266366
  -0.669689   -0.03589565  0.5764537  -1.7097796  -0.14665571  0.27364424
  -1.2197844  -0.17900002  0.81105083  0.49735194 -0.09490854  0.6097086
   0.13635202 -0.41592193 -0.5177906  -0.04319844  0.6383545  -0.5826326
  -0.08961103  0.6948251  -0.37232247 -0.10304417  0.00706872  0.81251115
   0.48310867  0.6950191  -0.22152555  0.08

In [43]:
np.savetxt('/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Domain_Adaptation/SentenceBERT/sentence_embeddings.txt', sentence_embeddings)

In [44]:
!pip install gpustat

In [45]:
!gpustat -cp

57e15a3dbd9d         Sat Feb  6 12:48:12 2021  418.67
[0] Tesla T4         | 45'C,   0 % |  2173 / 15079 MB |


In [46]:
from sentence_transformers import util

In [47]:
def community_detection(embeddings, threshold=0.75, min_community_size=10, init_max_size=1000):
    """
    Function for Fast Community Detection
    Finds in the embeddings all communities, i.e. embeddings that are close (closer than threshold).
    Returns only communities that are larger than min_community_size. The communities are returned
    in decreasing order. The first element in each list is the central point in the community.
    """

    # Compute cosine similarity scores
    cos_scores = util.pytorch_cos_sim(embeddings, embeddings)

    # Minimum size for a community
    top_k_values, _ = cos_scores.topk(k=min_community_size, largest=True)

    # Filter for rows >= min_threshold
    extracted_communities = []
    for i in range(len(top_k_values)):
        if top_k_values[i][-1] >= threshold:
            new_cluster = []

            # Only check top k most similar entries
            top_val_large, top_idx_large = cos_scores[i].topk(k=init_max_size, largest=True)
            top_idx_large = top_idx_large.tolist()
            top_val_large = top_val_large.tolist()

            if top_val_large[-1] < threshold:
                for idx, val in zip(top_idx_large, top_val_large):
                    if val < threshold:
                        break

                    new_cluster.append(idx)
            else:
                # Iterate over all entries (slow)
                for idx, val in enumerate(cos_scores[i].tolist()):
                    if val >= threshold:
                        new_cluster.append(idx)

            extracted_communities.append(new_cluster)

    # Largest cluster first
    extracted_communities = sorted(extracted_communities, key=lambda x: len(x), reverse=True)

    # Step 2) Remove overlapping communities
    unique_communities = []
    extracted_ids = set()

    for community in extracted_communities:
        add_cluster = True
        for idx in community:
            if idx in extracted_ids:
                add_cluster = False
                break

        if add_cluster:
            unique_communities.append(community)
            for idx in community:
                extracted_ids.add(idx)

    return unique_communities

In [48]:
samples = 20000

In [49]:
sentence_embeddings = np.loadtxt('/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Domain_Adaptation/SentenceBERT/sentence_embeddings.txt', max_rows =20000)

In [50]:
sentence_embeddings = sentence_embeddings[:samples]

In [51]:
print(sentence_embeddings.shape)

(20000, 768)


In [60]:
import time
print("Start clustering")
start_time = time.time()
clusters = community_detection(sentence_embeddings, min_community_size=40, threshold=0.95)
#Print all cluster / communities
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster:
        print("\t", sentences[sentence_id])

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Start clustering

Cluster 1, #757 Elements 
	 پذيرانه .
	 پويش .
	 دوفعه .
	 مرهم .
	 برات .
	 پربها .
	 ژستي .
	 ايان بسه
	 باجداري .
	 الکي .
	 مکتشف .
	 بداهه .
	 جاويد .
	 جاستين .
	 همگاه .
	 ريشه‌چه .
	 منسوب .
	 کيفم .
	 رجگر .
	 بيختن .
	 فاراده .
	 همديگر .
	 بي نزاکتي .
	 الساعه .
	 جايگاه .
	 وسمه .
	 الي ارتور .
	 بي فصل .
	 مغذي .
	 گزينشي .
	 هذلولي .
	 درباري .
	 مخل .
	 وليعهد
	 متحدالقول .
	 رجه .
	 يکدنده .
	 برخي .
	 که از آنجا .
	 اتيکت .
	 پيخال .
	 ...بانوي من
	 لفاظي .
	 به او .
	 بقال .
	 حالا ا .
	 فرديت .
	 گيوم وا
	 مانور .
	 ازبکي .
	 امري .
	 ستوان
	 سمينار .
	 فارقليط .
	 زبر تن .
	 لطايف .
	 گواراست
	 حفظ .
	 آوائي .
	 خيلي ناداني
	 عبارت .
	 ميدوني مثلا
	 ببالارفتن .
	 اتکاء .
	 تودرتوئي .
	 دغل .
	 چنبري .
	 سرگي .
	 چاک .
	 ...ميدوني
	 مستدعي .
	 دوپني .
	 خب ، بگو .
	 منتسب .
	 کاپي .
	 ، مانچو .
	 کمانگير .
	 گيشا .
	 غيررگي .
	 آزمندي .
	 نامهيا .
	 بخش .
	 گره .
	 پنجه .
	 Nنهo .
	 سر دسته .
	 کرانه زي .
	 ستوان .
	 کپي از کپي .
	 دريع داشتن .
	 اک